# Hand recognition - live demo
It's time to present you the effects of model training. So let's begin...

In [1]:
# Import PyTorch libraries 
import torch
import torchvision

# Load best_model file created in the train_model notebook
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 7)
model.load_state_dict(torch.load('best_model.pth'))
device = torch.device('cuda')
model = model.to(device)

# Time to preprocess the data
# Convert from BGR to RGB
# Normalize the data (our camera provides values in [0, 255] range and training loaded images in [0, 1])
# Transfer the data from CPU memory to GPU memory
# Add a batch dimension

import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

# If you think you already saw these lines - that's great! 
# We used them while creating widgets in data_collection notebook
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
go_slider = widgets.FloatSlider(description='go', min=0.0, max=1.0, orientation='vertical')
stop_slider = widgets.FloatSlider(description='stop', min=0.0, max=1.0, orientation='vertical')
left_slider = widgets.FloatSlider(description='left', min=0.0, max=1.0, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=0.0, max=1.0, orientation='vertical')
circle_slider = widgets.FloatSlider(description='circle', min=0.0, max=1.0, orientation='vertical')
free_slider = widgets.FloatSlider(description='free', min=0.0, max=1.0, orientation='vertical')
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, go_slider, stop_slider, left_slider, right_slider, circle_slider, free_slider, blocked_slider]))

# Let's import robot library to control the motors:
from jetbot import Robot

robot = Robot()

# The whole code below will update the status of the robot after each camera update
# It will preprocess the images, put them into network and get the results
# According to them proper movement will be executed

import torch.nn.functional as F
import time

acummullator = 0 # variable to control how many camera 
# frames in a row should contain the same network recognition
max_index = 0 # defines the index of the output (eg. circle = 0, free = 1)
set_go = 0
def update(change):
    global blocked_slider, robot, go_slider, stop_slider, left_slider, right_slider, circle_slider, free_slider, acummullator, max_index, set_go
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    #print(y)
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    speedvalues = [0.1, 0.2, 0.3, 0.4]
    prob_blocked = float(y.flatten()[0]) #the numbers in y.flatten[_] represent the output number, they're sorted alfabetically by folder name
    prob_go = float(y.flatten()[3])
    prob_stop = float(y.flatten()[6])
    prob_left = float(y.flatten()[4])
    prob_right = float(y.flatten()[5])
    prob_circle = float(y.flatten()[1])
    prob_free = float(y.flatten()[2]) 
    blocked_slider.value = prob_blocked
    go_slider.value = prob_go
    stop_slider.value = prob_stop
    left_slider.value = prob_left
    right_slider.value = prob_right
    circle_slider.value = prob_circle
    free_slider.value = prob_free
    temp_list = [prob_go, prob_stop, prob_left, prob_right, prob_circle, prob_blocked, prob_free]
    max_value = max(temp_list)
    max_index_tmp = temp_list.index(max_value)
    
    # robot logic of movement
    if max_index_tmp != max_index:
        acummullator = 1
    else:
        acummullator += 1
    max_index = max_index_tmp
    if acummullator > 5:
        if max_index == 0:      #go_prob
            robot.forward(0.4)
            set_go = 1
        elif max_index == 1:    #stop_prob
            robot.stop()
            set_go = 0
        elif max_index == 2:    #left_prob
            robot.left(0.4)
        elif max_index == 3:    #prob_right
            robot.right(0.4)
        elif max_index == 4:    #circle prob
            robot.right(0.5)
            time.sleep(0.2)
        elif max_index == 5:    #blocked prob
            robot.left(0.4)
        elif max_index == 6:    #prob_free
            if set_go == 1:
                robot.forward(0.4)
            elif set_go == 0:
                robot.stop()

    time.sleep(0.01) # we don't want the robot to clog up the robots calculation capabilities
        
update({'new': camera.value})  # we call the function once to intialize   

CPU times: user 17 s, sys: 7.64 s, total: 24.6 s
Wall time: 37.8 s


The function below will run the robot - **it will start moving!** Everytime we'll get the update frame from camera the function ``update`` will execute

In [3]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In the snippet below we can stop the execution of the update function everytime we get new frame of the camera, as well as stop the robot.

In [4]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

Remember to stop the camera if you want to run the whole code again, otherwise you'll have to reboot the jetbot after stopping the kernel.

In [5]:
camera.stop()

and here we have camera unlink

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)